In [94]:
import pandas as pd
import numpy as np

volumedf=pd.read_csv('D:/Amap/huaweidata/result0801/car_num.csv')
volumedf.fillna(0,inplace=True)
roaddf=pd.read_excel('D:/Amap/processdata/路网与边界/wuhan/osm_gcj_5659_xy12/roadlength.xlsx')
roaddf.rename(columns={'idnew':'road_id'},inplace=True)
mergedf=pd.merge(volumedf,roaddf,on='road_id',how='left')
mergedf.iloc[:, 1:-1] = mergedf.iloc[:, 1:-1].div(mergedf.iloc[:, -1], axis=0)
volumeResult=mergedf.iloc[:,0:289]
volumeResult.set_index('road_id',inplace=True)

speeddf=pd.read_csv('D:/Amap/huaweidata/result0801/2023_08_01_speed.csv')
speeddf.fillna(0,inplace=True)

speeddf.rename(columns={'objectID':'road_id'},inplace=True)
speeddf.set_index('road_id',inplace=True)



from scipy.optimize import curve_fit

# 定义一元二次方程
def quadratic_function(x, a, b):
    return a * x + b

# 拟合数据并存储结果
result = []
for index, row in speeddf.iterrows():
    road_id=index
    speed=row.values
    volume=volumeResult.loc[index,:].values
    new=[road_id,speed,volume]
    result.append(new)

resultdf=pd.DataFrame(result,columns=['road_id','speed','volume'])
df=resultdf


import pandas as pd
import numpy as np
from scipy.optimize import curve_fit

# 假设您的数据存储在一个名为df的DataFrame中，包含列：road_id，speed，volume

# 定义二次方程模型
def quadratic_model(x, a, b):
    return b * x + a

for index, row in df.iterrows():
    road_id = row['road_id']
    speed = np.array(row['speed'])
    volume = np.array(row['volume'])

    # 移除volume为0的数据点
    non_zero_indices = np.nonzero(volume)
    speed = speed[non_zero_indices]
    volume = volume[non_zero_indices]

    # 如果还有数据点，对speed拟合二次方程并存储系数
    if len(speed) > 0:
        # 使用curve_fit拟合带有约束条件的二次方程
        popt, pcov = curve_fit(quadratic_model, volume, speed, bounds=([0, -np.inf], np.inf))

        # 提取拟合的系数
        a_speed, b_speed = popt

        # 将结果追加到新的DataFrame
        result_df = result_df.append({'road_id': road_id, 'a': a_speed, 'b': b_speed}, ignore_index=True)

# 显示结果DataFrame
print(result_df)

In [110]:
#index
df=speeddf
df.columns = pd.to_datetime(df.columns, format='%Y%m%d%H%M')

# 选择时间在6:00到24:00之间的列
start_time = pd.to_datetime('6:00', format='%H:%M').time()
end_time = pd.to_datetime('23:59', format='%H:%M').time()

selected_columns = df.columns[(df.columns.time >= start_time) & (df.columns.time <= end_time)]

# 提取所选列和road_id列
selected_df = df[ selected_columns.tolist()]
selected_df=selected_df.transpose()

# 遍历每一列，排序并计算前1/9的数据的平均值
for column in selected_df.columns[0:]:  # Skipping the first column 'road_id'
    # 对每一列进行排序
    selected_df[column] = selected_df[column].sort_values().values

nineVdf=selected_df.iloc[-24:,:]
nineVdf=nineVdf.transpose()
nineVdf['mean']=nineVdf.mean(axis=1)
final=nineVdf[['mean']]
final.to_csv('result.csv')